In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# Driver path
path = '/Users/kurum/Downloads/chromedriver.exe'
service = Service(path)
driver = webdriver.Chrome(service=service)

# Maximize Window
driver.maximize_window()
driver.minimize_window()
driver.maximize_window()
driver.switch_to.window(driver.current_window_handle)
driver.implicitly_wait(10)

# Rest of the code remains the same

# Enter the site
driver.get('https://www.linkedin.com/login')
time.sleep(2)

# User Credentials
user_name = '......'
password = '....'
driver.find_element(By.XPATH, '//*[@id="username"]').send_keys(user_name)
driver.find_element(By.XPATH, '//*[@id="password"]').send_keys(password)
time.sleep(1)

# Login button
driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button').click()
driver.implicitly_wait(30)

# Jobs page
driver.find_element(By.XPATH, '//*[@id="global-nav"]/div/nav/ul/li[3]/a/span').click()
time.sleep(3)
# Go to search results directly
driver.get("https://www.linkedin.com/jobs/search/?currentJobId=3568598652&f_E=3%2C4%2C5%2C6&f_TPR=r86400&geoId=103644278&keywords=(GPT%20OR%20LLM%20OR%20Generative%20AI)%20AND%20(Machine%20%20OR%20ML%20OR%20Data%20Science%20OR%20AI%20OR%20Software)&location=United%20States&refresh=true&start=50")
time.sleep(1)

# Get all links for these offers
links = []
# Navigate multiple pages
# Navigate to the last page
last_page_button = driver.find_element(By.CSS_SELECTOR, 'li.artdeco-pagination__indicator--number:last-child button')
last_page_number = int(last_page_button.text)

print(f'The last page number is: {last_page_number}')

# Adjust the loop range

print('Links are being collected now.')
try:
    for page in range(1, last_page_number + 1):
        time.sleep(2)
        jobs_block = driver.find_element(By.CLASS_NAME, 'jobs-search-results-list')
        jobs_list = jobs_block.find_elements(By.CSS_SELECTOR, 'li.jobs-search-results__list-item')

        for job in jobs_list:
            all_links = job.find_elements(By.TAG_NAME, 'a')
            for a in all_links:
                if str(a.get_attribute('href')).startswith("https://www.linkedin.com/jobs/view") and a.get_attribute('href') not in links:
                    links.append(a.get_attribute('href'))
                else:
                    pass
            # Scroll down for each job element
            driver.execute_script("arguments[0].scrollIntoView();", job)

        print(f'Collecting the links in the page: {page - 1}')
        # Go to next page
       # Construct the CSS selector for the current page button
        page_button_selector = f"li.artdeco-pagination__indicator--number button[aria-label='Page {page}']"

       # Wait for the page button to be clickable
        page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, page_button_selector)))

       # Scroll to the page button element
        driver.execute_script("arguments[0].scrollIntoView();", page_button)

       # Click the page button
        page_button.click()

       # Wait for the page to load
        time.sleep(3)
        
except:
    pass
print('Found ' + str(len(links)) + ' links for job offers')

# Create empty lists to store information
job_titles = []
company_names = []
company_locations = []
work_methods = []
post_dates = []
work_times = []
job_desc = []

i = 0
j = 1
# Visit each link one by one to scrape the information
print('Visiting the links and collecting information just started.')
for i in range(len(links)):
    try:
        driver.get(links[i])
        i = i + 1
        time.sleep(2)
        # Click See more.
        driver.find_element(By.CLASS_NAME, "artdeco-card__actions").click()
        time.sleep(2)
    except:
        pass

    # Find the general information of the job offers
    contents = driver.find_elements(By.CLASS_NAME, 'p5')
    for content in contents:
        try:
            job_titles.append(content.find_element(By.TAG_NAME, "h1").text)
            company_names.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").text)
            company_locations.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").text)
            work_methods.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").text)
            post_dates.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").text)
            work_times.append(content.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-insight").text)
            print(f'Scraping the Job Offer {j} DONE.')
            j += 1

        except:
            pass
        time.sleep(2)


# Creating the dataframe
df = pd.DataFrame(
    list(zip(job_titles, company_names, company_locations, work_methods, post_dates, work_times)),
    columns=['job_title', 'company_name', 'company_location', 'work_method', 'post_date', 'work_time'])
df.head()

# Storing the data to excel file
df.to_excel(r'Path of excel\File Name.xlsx', sheet_name='Your sheet name', index=False)


